In [45]:
import os
import numpy as np
import nibabel as nib
import tensorflow as tf
import flwr as fl
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import matplotlib.pyplot as plt
from multiprocessing import Process


In [26]:
# Configurations
IMG_SIZE = 128
VOLUME_SLICES = 50
VOLUME_START_AT = 22
BATCH_SIZE = 8
NUM_EPOCHS = 5
NUM_ROUNDS = 5


In [46]:
# Dataset path
images_path = "Data/BraTS2021"

# Load all patients
all_patients = [os.path.join(images_path, p) for p in os.listdir(images_path) if not p.startswith(".")]
np.random.shuffle(all_patients)

# Split dataset across hospitals
datasets = {
    "1": all_patients[:len(all_patients)//3],
    "2": all_patients[len(all_patients)//3:2*len(all_patients)//3],
    "3": all_patients[2*len(all_patients)//3:]
}

In [54]:
from tensorflow import keras

# Define Data Generator
class DataGenerator(keras.utils.Sequence):
    def __init__(self, list_IDs, batch_size=1, dim=(IMG_SIZE, IMG_SIZE), n_channels=2, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        Batch_ids = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(Batch_ids)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, Batch_ids):
        X = np.zeros((self.batch_size * VOLUME_SLICES, *self.dim, self.n_channels))
        y = np.zeros((self.batch_size * VOLUME_SLICES, IMG_SIZE, IMG_SIZE))

        for c, i in enumerate(Batch_ids):
            case_path = os.path.join(images_path, os.path.basename(i), os.path.basename(i))
            try:
                flair = nib.load(f'{case_path}_flair.nii').get_fdata()
                ce = nib.load(f'{case_path}_t1ce.nii').get_fdata()
                seg = nib.load(f'{case_path}_seg.nii').get_fdata()

                for j in range(VOLUME_SLICES):
                    X[j + VOLUME_SLICES * c, :, :, 0] = cv2.resize(flair[:, :, j + VOLUME_START_AT], (IMG_SIZE, IMG_SIZE))
                    X[j + VOLUME_SLICES * c, :, :, 1] = cv2.resize(ce[:, :, j + VOLUME_START_AT], (IMG_SIZE, IMG_SIZE))
                    y[j + VOLUME_SLICES * c] = cv2.resize(seg[:, :, j + VOLUME_START_AT], (IMG_SIZE, IMG_SIZE))

                y[y == 4] = 3  # Adjust class values
                mask = tf.one_hot(y, 4)
                Y = tf.image.resize(mask, (IMG_SIZE, IMG_SIZE), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

                return X / np.max(X), Y
            
            except Exception as e:
                print(f"Error loading file {case_path}: {e}")
                return np.zeros_like(X), np.zeros_like(Y)

# Assign DataGenerators to Federated Clients
hospital_generators = {
    "1": DataGenerator(datasets["1"], batch_size=BATCH_SIZE),
    "2": DataGenerator(datasets["2"], batch_size=BATCH_SIZE),
    "3": DataGenerator(datasets["3"], batch_size=BATCH_SIZE),
}


In [ ]:
# Check if dataset loading is working correctly for Hospital 1
x_batch, y_batch = hospital_generators["1"].__getitem__(0)

print("Dataset Loading Verification:")
print(f"Input MRI Shape: {x_batch.shape}")
print(f"Segmentation Mask Shape: {y_batch.shape}")
print(f"Max Intensity Value in Input: {np.max(x_batch)}")
print(f"Unique Labels in Segmentation: {np.unique(y_batch)}")

# Visualize one MRI slice and its corresponding segmentation mask
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.imshow(x_batch[0, :, :, 0], cmap="gray")
plt.title("FLAIR MRI Slice")

plt.subplot(1, 2, 2)
plt.imshow(np.argmax(y_batch[0], axis=-1), cmap="jet")  # Convert one-hot to categorical
plt.title("Ground Truth Segmentation Mask")

plt.show()


In [56]:
def create_unet_model(input_shape=(128, 128, 2)):  
    inputs = layers.Input(input_shape)
    conv1 = layers.Conv2D(64, 3, activation="relu", padding="same")(inputs)
    conv1 = layers.Conv2D(64, 3, activation="relu", padding="same")(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation="relu", padding="same")(pool1)
    conv2 = layers.Conv2D(128, 3, activation="relu", padding="same")(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation="relu", padding="same")(pool2)

    up1 = layers.UpSampling2D(size=(2, 2))(conv3)
    up1 = layers.Conv2D(128, 3, activation="relu", padding="same")(up1)
    up1 = layers.Concatenate()([conv2, up1])

    up2 = layers.UpSampling2D(size=(2, 2))(up1)
    up2 = layers.Conv2D(64, 3, activation="relu", padding="same")(up2)
    up2 = layers.Concatenate()([conv1, up2])

    outputs = layers.Conv2D(1, 1, activation="sigmoid", padding="same")(up2)

    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model


In [64]:
class HospitalClient(fl.client.NumPyClient):
    def __init__(self, model, train_generator, test_generator):
        self.model = model
        self.train_generator = train_generator
        self.test_generator = test_generator

    def get_parameters(self):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        
        print(f"\n🏥 [FL Client] Training Model for Hospital...\n")
        
        history = self.model.fit(self.train_generator, epochs=1, verbose=1)
        
        print(f"✔ [FL Training Done] Loss: {history.history['loss'][-1]:.4f}, Accuracy: {history.history['accuracy'][-1]:.4f}")
        
        return self.model.get_weights(), len(self.train_generator), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.test_generator, verbose=0)
        print(f"📊 [FL Evaluation] Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
        return loss, len(self.test_generator), {"accuracy": accuracy}

# Assign Clients
hospital_clients = {
    "1": HospitalClient(create_unet_model(), hospital_generators["1"], hospital_generators["1"]),
    "2": HospitalClient(create_unet_model(), hospital_generators["2"], hospital_generators["2"]),
    "3": HospitalClient(create_unet_model(), hospital_generators["3"], hospital_generators["3"]),
}

In [68]:
from multiprocessing import Process

def start_server():
    print("\n🚀 [FL Server] Starting Federated Learning...\n")

    def fit_round(server_round, parameters, config):
        print(f"\n📡 [FL Server] Received Model Update for Round {server_round}\n")
        return parameters, {}

    strategy = fl.server.strategy.FedAvg(on_fit_config_fn=fit_round)
    
    fl.server.start_server(config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS), strategy=strategy)
    
    print("\n✅ [FL Server] Training Completed!")



def start_client(hospital_id):
    print(f"\n🚀 [Client {hospital_id}] Attempting to Connect to FL Server...\n")
    
    client = hospital_clients[hospital_id]
    
    print(f"\n✅ [Client {hospital_id}] Successfully Started\n")
    
    fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=client)
    
    print(f"\n🏥 [Client {hospital_id}] Finished Training!\n")


# Run FL Training and Observe Output

# Launch server and clients
server_process = Process(target=start_server)
server_process.start()

client_processes = []
p = Process(target=start_client, args=("1",))  # Only run Client 1
client_processes.append(p)
p.start()

p.join()  # Ensure the client finishes before moving to the next one

server_process.terminate()


In [ ]:
import tensorflow.keras.backend as K

# Dice Coefficient Metric
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# IoU Metric
def iou(y_true, y_pred, smooth=1e-6):
    intersection = K.sum(y_true * y_pred)
    total = K.sum(y_true) + K.sum(y_pred)
    union = total - intersection
    return (intersection + smooth) / (union + smooth)

# Evaluate Model Performance Across Hospitals
hospital_scores = {}
for hospital_id in ["1", "2", "3"]:
    print(f"Evaluating Model for Hospital {hospital_id}...")
    
    x_test, y_test = hospital_generators[hospital_id].__getitem__(0)
    
    y_pred = hospital_clients[hospital_id].model.predict(x_test)
    dice = dice_coefficient(y_test, y_pred)
    iou_score = iou(y_test, y_pred)
    
    hospital_scores[hospital_id] = {"Dice": K.eval(dice), "IoU": K.eval(iou_score)}

print("\nFederated Learning Model Performance Across Hospitals:")
for hospital, scores in hospital_scores.items():
    print(f"Hospital {hospital} -> Dice: {scores['Dice']:.4f}, IoU: {scores['IoU']:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Visualize segmentation outputs
def visualize_segmentation(model, x_test, y_test, index=0):
    prediction = model.predict(x_test[index:index+1])[0, :, :, 0]
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 3, 1)
    plt.title("Input Image (FLAIR)")
    plt.imshow(x_test[index, :, :, 0], cmap="gray")
    
    plt.subplot(1, 3, 2)
    plt.title("Ground Truth")
    plt.imshow(y_test[index, :, :, 0], cmap="gray")
    
    plt.subplot(1, 3, 3)
    plt.title("Predicted Segmentation")
    plt.imshow(prediction, cmap="gray")
    
    plt.show()

# Select a hospital and visualize segmentation
x_test, y_test = next(iter(create_dataset(datasets["1"])))
visualize_segmentation(global_model, x_test, y_test, index=0)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Conv2D

# Prepare tumor progression dataset
def prepare_tumor_time_series(file_paths, time_steps=5):
    sequences = []
    for file_path in file_paths:
        # Load segmented MRI slices
        seg = nib.load(file_path + '_seg.nii').get_fdata()
        seg = cv2.resize(seg, (IMG_SIZE, IMG_SIZE))

        # Normalize & reshape
        seg = seg / np.max(seg)
        seq = [seg[:, :, i] for i in range(time_steps)]
        sequences.append(seq)
    
    return np.array(sequences)

# Prepare dataset for LSTM training
X_tumor_train = prepare_tumor_time_series(datasets["1"], time_steps=5)
Y_tumor_train = X_tumor_train[:, 1:]  # Next MRI slice is target


In [ ]:
# Define LSTM-based tumor growth predictor
model_tumor = Sequential([
    LSTM(128, return_sequences=True, input_shape=(4, IMG_SIZE*IMG_SIZE)),
    LSTM(64, return_sequences=False),
    Dense(IMG_SIZE*IMG_SIZE, activation="relu"),  # Predict next MRI slice
])

model_tumor.compile(optimizer="adam", loss="mse")
model_tumor.fit(X_tumor_train, Y_tumor_train, epochs=50, batch_size=4, verbose=1)


In [ ]:
# Predict Next MRI Slice (Tumor Evolution)
def predict_tumor_growth(model, x_test):
    pred = model.predict(x_test)
    return pred.reshape(IMG_SIZE, IMG_SIZE)

# Select a test patient
x_tumor_test = X_tumor_train[0:1]
predicted_tumor = predict_tumor_growth(model_tumor, x_tumor_test)

# Visualize Tumor Evolution Prediction
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title("Current Tumor MRI Slice")
plt.imshow(x_tumor_test[0, -1], cmap="gray")

plt.subplot(1, 2, 2)
plt.title("Predicted Future Tumor Growth")
plt.imshow(predicted_tumor, cmap="gray")
plt.show()
